In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import json
import joblib
import os

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/213141
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Set HDFS paths for the datasets
traffic_trainset_path = r'\traffic_trainset_hour.csv'
weather_trainset_path = r'\weather_trainset.csv'
traffic_testset_path = r'\traffic_testset.csv'
weather_testset_path = r'\weather_testset.csv'

# Load datasets
dataset_path = os.getcwd()+r'\historical_dataset'
traffic_train = pd.read_csv(dataset_path+traffic_trainset_path)
weather_train = pd.read_csv(dataset_path+weather_trainset_path)
traffic_test = pd.read_csv(dataset_path+traffic_testset_path)
weather_test = pd.read_csv(dataset_path+weather_testset_path)

In [24]:
weather_test['day'] = 29

In [26]:
# Join training datasets on ['day', 'hour'] and select features
useless_columns = ['free_flow_speed', 'confidence', 'weekend', 'day']
training_data_merged = pd.merge(traffic_train, weather_train, on=['day', 'hour'])
training_data = training_data_merged.drop(columns=useless_columns)

# Join test datasets on ['day', 'hour'] and select features

test_data_merged = pd.merge(traffic_test, weather_test, on=['day', 'hour'])
test_data = test_data_merged.drop(columns=useless_columns)

In [29]:
# Split data into features (X) and target variable (y)

X_train = training_data.drop('current_speed', axis=1)
y_train = training_data['current_speed']

X_test = test_data.drop('current_speed', axis=1)
y_test = test_data['current_speed']

In [1]:
X_test

NameError: name 'X_test' is not defined

In [30]:
# Model training and evaluation

# K-Nearest Neighbors
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)
knn_rmse = mean_squared_error(y_test, knn_predictions, squared=False)
knn_r2 = r2_score(y_test, knn_predictions)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
lr_rmse = mean_squared_error(y_test, lr_predictions, squared=False)
lr_r2 = r2_score(y_test, lr_predictions)

# AdaBoost Regression
ada_model = AdaBoostRegressor()
ada_model.fit(X_train, y_train)
ada_predictions = ada_model.predict(X_test)
ada_rmse = mean_squared_error(y_test, ada_predictions, squared=False)
ada_r2 = r2_score(y_test, ada_predictions)

# XGBoost Regression
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
xgb_r2 = r2_score(y_test, xgb_predictions)

In [31]:
# Model comparison

model_metrics = pd.DataFrame({
    'Model': ['KNN', 'Linear Regression', 'AdaBoost Regression', 'XGBoost Regression'],
    'RMSE': [knn_rmse, lr_rmse, ada_rmse, xgb_rmse],
    'R2 Score': [knn_r2, lr_r2, ada_r2, xgb_r2]
})

best_model = model_metrics.loc[model_metrics['RMSE'].idxmin()]

print("Model Comparison:")
print(model_metrics)
print("\nBest Model:")
print(best_model)

Model Comparison:
                 Model      RMSE  R2 Score
0                  KNN  3.259601  0.600287
1    Linear Regression  4.526025  0.229358
2  AdaBoost Regression  2.599834  0.745721
3   XGBoost Regression  2.722941  0.721070

Best Model:
Model       AdaBoost Regression
RMSE                   2.599834
R2 Score               0.745721
Name: 2, dtype: object


In [32]:
# Save the best model and performance metrics to a directory
mr = project.get_model_registry()

# Create a directory for the model
model_dir="traffic_weather_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save KNN model
ada_model_path = model_dir + '/ada_model.joblib'
joblib.dump(ada_model, ada_model_path)

# Save performance metrics
metrics_path = model_dir + '/model_metrics.json'
model_metrics.to_json(metrics_path, orient='records')

print("Model and metrics saved to:", model_dir)


Connected. Call `.close()` to terminate connection gracefully.
Model and metrics saved to: traffic_weather_model


In [33]:
# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry

traffic_weather_model = mr.python.create_model(
    name="traffic_weather_model", 
    metrics={
        'RMSE': float(best_model['RMSE']),
        'R2 Score': float(best_model['R2 Score'])},
    model_schema=model_schema,
    description="Traffic Flow Predictor"
)
             
             
# Upload the model to the model registry, including all files in 'model_dir'
traffic_weather_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/50336 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/285 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/692 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/213141/models/traffic_weather_model/1


Model(name: 'traffic_weather_model', version: 1)